In [ ]:
import time
import math

class Node:
    def __init__(self, key):
        self.key = key
        self.parent = None
        self.child = None
        self.left = self
        self.right = self
        self.degree = 0
        self.mark = False

class FibonacciHeap:
    def __init__(self):
        self.min_node = None
        self.total_nodes = 0

    def insert(self, key):
        node = Node(key)
        if self.min_node is None:
            self.min_node = node
        else:
            self._merge_with_root_list(self.min_node, node)
            if node.key < self.min_node.key:
                self.min_node = node
        self.total_nodes += 1
        return node

    def _merge_with_root_list(self, a, b):
        a.right.left = b.left
        b.left.right = a.right
        a.right = b
        b.left = a

    def find_min(self):
        return self.min_node.key if self.min_node else None

    def extract_min(self):
        z = self.min_node
        if z is not None:
            if z.child is not None:
                children = [x for x in self._iterate(z.child)]
                for child in children:
                    self._merge_with_root_list(self.min_node, child)
                    child.parent = None
            z.left.right = z.right
            z.right.left = z.left
            if z == z.right:
                self.min_node = None
            else:
                self.min_node = z.right
                self._consolidate()
            self.total_nodes -= 1
        return z

    def _iterate(self, head):
        node = stop = head
        flag = False
        while True:
            if node == stop and flag:
                break
            elif node == stop:
                flag = True
            yield node
            node = node.right

    def _consolidate(self):
        A = [None] * int(math.log(self.total_nodes + 1) * 2)
        root_nodes = [x for x in self._iterate(self.min_node)]
        for w in root_nodes:
            x = w
            d = x.degree
            while A[d] is not None:
                y = A[d]
                if x.key > y.key:
                    x, y = y, x
                self._heap_link(y, x)
                A[d] = None
                d += 1
            A[d] = x
        self.min_node = None
        for i in range(len(A)):
            if A[i] is not None:
                if self.min_node is None:
                    self.min_node = A[i]
                else:
                    self._merge_with_root_list(self.min_node, A[i])
                    if A[i].key < self.min_node.key:
                        self.min_node = A[i]

    def _heap_link(self, y, x):
        y.left.right = y.right
        y.right.left = y.left
        y.parent = x
        if x.child is None:
            x.child = y
            y.left = y.right = y
        else:
            self._merge_with_root_list(x.child, y)
        x.degree += 1
        y.mark = False

    def decrease_key(self, x, k):
        if k > x.key:
            raise ValueError("new key is greater than current key")
        x.key = k
        y = x.parent
        if y is not None and x.key < y.key:
            self._cut(x, y)
            self._cascading_cut(y)
        if x.key < self.min_node.key:
            self.min_node = x

    def _cut(self, x, y):
        if x.right == x:
            y.child = None
        else:
            x.right.left = x.left
            x.left.right = x.right
            if y.child == x:
                y.child = x.right
        y.degree -= 1
        self._merge_with_root_list(self.min_node, x)
        x.parent = None
        x.mark = False

    def _cascading_cut(self, y):
        z = y.parent
        if z is not None:
            if not y.mark:
                y.mark = True
            else:
                self._cut(y, z)
                self._cascading_cut(z)

def run_benchmark():
    heap = FibonacciHeap()
    inserted_nodes = []
    n = 1000

    start = time.time()
    for i in range(n):
        inserted_nodes.append(heap.insert(i))
    insert_time = (time.time() - start) / n * 1000

    start = time.time()
    for i in range(n // 2):
        heap.decrease_key(inserted_nodes[n - 1 - i], -i)
    decrease_key_time = (time.time() - start) / (n // 2) * 1000

    start = time.time()
    for _ in range(n):
        heap.extract_min()
    extract_min_time = (time.time() - start) / n * 1000

    print(f"Insert: {insert_time:.5f} ms/op")
    print(f"Decrease-Key: {decrease_key_time:.5f} ms/op")
    print(f"Extract-Min: {extract_min_time:.5f} ms/op")

if __name__ == "__main__":
    run_benchmark()
